# Using base64 encoded file for the Object Detection

---

## Overall trials

### Load required libraries

In [ ]:
import base64
import torch
import os

from PIL import Image
from PIL import Image as im
from torchvision import models, transforms

### Checking current directory

In [ ]:
direct = os.getcwd()
print(direct)

### Load base64 encoded txt file

In [ ]:
## load and read base64.txt file
file = open(direct + '/base_image.txt', 'rb')
encoded_data = file.read()
file.close()

### base64 decode to jpeg format

In [ ]:
#decode base64 string data
decoded_data=base64.b64decode((encoded_data))
#write the decoded data back to original format in  file
img_file = open(direct + '/decoded_image.jpeg', 'wb')
img_file.write(decoded_data)
img_file.close()

In [ ]:
# import numpy as np
# import io, cv2

# def stringToRGB(base64_string):
#     imgdata = base64.b64decode(str(base64_string))
#     img = Image.open(io.BytesIO(imgdata))
#     opencv_img= cv2.cvtColor(np.array(img), cv2.COLOR_BGR2RGB)
#     return opencv_img

# print(type(stringToRGB(encoded_data)))
# img = im.fromarray(stringToRGB(encoded_data))

### Prepare the image for inference

In [ ]:
## load image for inference
img = Image.open(direct + "/decoded_image.jpeg")

In [ ]:
## data transform code for inference
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

In [ ]:
## image transform and batch setup
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)

### Prepare pre-trained model (here, mobilenet_v3_small)

In [ ]:
## prepare pretrained model (trained on Imagenet-1k) and model evaluation
model = models.mobilenet_v3_small(pretrained=True)
model.eval()

### Inference

In [ ]:
## input data inference
out = model(batch_t)

In [ ]:
## prepare candidate labels(classes)
with open('/content/imagenet-classes.txt') as f:
    classes = [line.strip() for line in f.readlines()]

In [ ]:
_, indices = torch.sort(out, descending=True)
percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100
[(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]
# print([(classes[idx]) for idx in indices[0][:3]])

----

## Summarized Code

In [ ]:
import base64
import torch
import os

from PIL import Image
from PIL import Image as im
from torchvision import models, transforms

def main(base_data):
    dir = os.getcwd()
    decoded_data=base64.b64decode((base_data))
    img_file = open(dir + '/decoded_image.jpeg', 'wb')
    img_file.write(decoded_data)
    img_file.close()

    img = Image.open(dir + "/decoded_image.jpeg")
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        )
    ])

    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)

    model = models.mobilenet_v3_small(pretrained=True)
    model.eval()
    out = model(batch_t)
    with open(dir + '/imagenet-classes.txt') as f:
        classes = [line.strip() for line in f.readlines()]

    _, indices = torch.sort(out, descending=True)
    percentage = torch.nn.functional.softmax(out, dim=1)[0] * 10
    print([(classes[idx]) for idx in indices[0][:3]])

In [ ]:
main(encoded_data)

---

## For API calls

In [ ]:
import base64
import torch
import os

from PIL import Image
from PIL import Image as im
from torchvision import models, transforms
from flask import Flask, request, jsonify


@base_classification.route('/obj', methods=['POST'])
base_classification = Flask(__name__)

def obj():
    if request.method =="POST":
        data = request.get_json()
        base = data['base64']
        
        
        dir = os.getcwd()
        decoded_data=base64.b64decode((base))
        img_file = open(dir + '/decoded_image.jpeg', 'wb')
        img_file.write(decoded_data)
        img_file.close()
        
        img = Image.open(dir + "/decoded_image.jpeg")
        transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
            )
        ])
        
        img_t = transform(img)
        batch_t = torch.unsqueeze(img_t, 0)
        
        model = models.mobilenet_v3_small(pretrained=True)
        model.eval()
        out = model(batch_t)
        with open(dir + '/imagenet-classes.txt') as f:
            classes = [line.strip() for line in f.readlines()]
            
            
        _, indices = torch.sort(out, descending=True)
        percentage = torch.nn.functional.softmax(out, dim=1)[0] * 10
        results = [(classes[idx]) for idx in indices[0][:3]]
        
        labels = {results}
        return jsonify({'labels': labels})
    
    else:
        return 'Invalid request method. Only POST is allowed.', 405


    
if __name__ == '__main__':
    base_classification.run(debug=True)